In [ ]:
import os
import torch
import numpy as np
import setup.dataset as dataset
import setup.UNet as UNet
import setup.ResUNet as ResUNet
import setup.classifier as classifier
import setup.plot as plot
from torch.utils.data import SubsetRandomSampler

np.random.seed(0)
torch.manual_seed(0)

DATASET_PATH = '/home/tungdao/Tung/code/ducanh/data/png_dataset'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def sampler_indices(length):
    indices = list(range(length))
    np.random.shuffle(indices)
    split = int(np.floor(0.1 * length))
    test_indices = indices[:split]
    return test_indices

tumor_dataset = dataset.TumorDataset(DATASET_PATH)

test_indices = sampler_indices(len(tumor_dataset))
test_sampler = SubsetRandomSampler(test_indices)

test_loader = torch.utils.data.DataLoader(tumor_dataset, batch_size=1, sampler=test_sampler)

FILTER_LIST = [16,32,64,128,256]

#model = UNet.DynamicUNet(FILTER_LIST).to(device)
#path = 'outputs/UNet.pt'
model = ResUNet.ResUNet(FILTER_LIST).to(device)
path = 'outputs/ResUNet.pt'

classifier = classifier.TumorClassifier(model, device)
if device == 'cpu':
    classifier.model.load_state_dict(torch.load(path, map_location='cpu'))
else:
    classifier.model.load_state_dict(torch.load(path))

In [ ]:
i=0

In [ ]:
image_index = test_indices[i]
sample = tumor_dataset[image_index]
image, mask, output, d_score = classifier.predict(sample,0.65)
title = f'Name: {image_index}.png   Dice Score: {d_score:.5f}'
plot.result(image,mask,output,title)
i += 1
if i >= len(test_indices):
    i = 0 